In [1]:
import numpy as np
from ase.build import bulk
from ase.calculators.emt import EMT
from phono3py import Phono3py

from vibes.structure.convert import to_phonopy_atoms, to_Atoms

In [2]:
atoms = bulk('Al', cubic=True)

smatrix = np.diag([2, 2, 2])
q_mesh = 3 * [5, ]

In [3]:
calc = EMT()

In [4]:
phonon3 = Phono3py(
    to_phonopy_atoms(atoms), supercell_matrix=smatrix, mesh=q_mesh, log_level=2
)

In [5]:
phonon3.generate_displacements(cutoff_pair_distance=5)

In [6]:
primitive = to_Atoms(phonon3.primitive)
supercell = to_Atoms(phonon3.supercell)

primitive

Atoms(symbols='Al4', pbc=True, cell=[4.05, 4.05, 4.05], masses=...)

In [7]:
scs = phonon3.get_supercells_with_displacements()

In [8]:
force_sets = []
for pa in scs:
    if pa is None:
        force_sets.append(np.zeros([len(supercell), 3]))
        continue
    a = to_Atoms(pa)
    a.calc = EMT()
    force_sets.append(a.get_forces())

In [9]:
phonon3.forces = np.array(force_sets)

In [10]:
phonon3.produce_fc2()
phonon3.produce_fc3()

Computing fc3[ 1, x, x ] using numpy.linalg.pinv with a displacement:
    [ 0.0300  0.0000  0.0000]
Expanding fc3
Cutting-off fc3 (cut-off distance: 5.000000)
Building atom mapping table...
Creating contracted fc3...


In [11]:
phonon3.run_thermal_conductivity(write_kappa=True)

-------------------- Lattice thermal conducitivity (RTA) --------------------
Lifetime sampling mesh: [ 5 5 5 ]
======================= Grid point 0 (1/10) =======================
q-point: ( 0.00  0.00  0.00)
Number of triplets: 10
Frequency     group velocity (x, y, z)     |gv|
   0.000   (   0.000    0.000    0.000)    0.000
   0.000   (   0.000    0.000    0.000)    0.000
   0.000   (   0.000    0.000    0.000)    0.000
   5.288   (   0.000    0.000    0.000)    0.000
   5.288   (   0.000    0.000    0.000)    0.000
   5.288   (   0.000    0.000    0.000)    0.000
   5.288   (   0.000    0.000    0.000)    0.000
   5.288   (   0.000    0.000    0.000)    0.000
   5.288   (   0.000    0.000    0.000)    0.000
   7.992   (   0.000    0.000    0.000)    0.000
   7.992   (   0.000    0.000    0.000)    0.000
   7.992   (   0.000    0.000    0.000)    0.000
======================= Grid point 1 (2/10) =======================
q-point: ( 0.20  0.00  0.00)
Number of triplets: 18
Frequency   